# Question 1

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 4000

nhl_df = pd.read_csv("./Course1_Resources/assignments/assignment4/assets/nhl.csv")
nba_df = pd.read_csv("./Course1_Resources/assignments/assignment4/assets/nba.csv")
mlb_df = pd.read_csv("./Course1_Resources/assignments/assignment4/assets/mlb.csv")
nfl_df = pd.read_csv("./Course1_Resources/assignments/assignment4/assets/nfl.csv")
cities = pd.read_html("./Course1_Resources/assignments/assignment4/assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]

In [25]:
# ------------- nhl_df -------------

# --- Names cleaning ---
# Pull the teams just fo 2018
teams = nhl_df[nhl_df['year'] == 2018]['team']

# Split the names for later match
teams = teams.str.extractall('([\w]*)(\W)?\s?').unstack()[0]

# Remove the NaN columns
teams = teams[[0, 1, 2]]

# Remove the 'Divisions'
teams = teams[teams[1] != 'Division']

# Replace NaN with empty strings
teams = teams.replace(np.nan, '')

# Create and append a list with the last name of the team for later match with cities df
team = []
index = []

for ind, row in teams.iterrows():
    if row[2] != '':
        team.append(row[2])
        index.append(ind)
    else:
        team.append(row[1])
        index.append(ind)

new = pd.DataFrame(team, index)
n_teams = pd.merge(teams, new, how = 'left', left_index = True, right_index = True)
n_teams.columns = [0, 1, 2, 'Matchname']
n_teams

# --- Putting together with the rest of the data ---
# Recovering data from wins and losses
wins_losses = nhl_df[nhl_df['year'] == 2018][['W', 'L']]
wins_losses = wins_losses.drop([0, 9, 18, 26], axis = 0)

# Merging both dataframes
df = pd.merge(n_teams, wins_losses, how = 'left', left_index = True, right_index = True)

# Convert columns W & L to numbers
df['Wint'] = [int(x) for x in df['W']]
df['Lint'] = [int(x) for x in df['L']]
df['W/L'] = df['Wint'] / (df['Wint'] + df['Lint'])
df.set_index('Matchname', inplace = True)


# ------------- cities -------------
# Pulling the names
teams_c = cities['NHL'].str.extractall('([A-Z][a-z]+)').unstack()

# Recovering population data
# I rename the column for easier handle
cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})
# I store the data of interest in a new df: population
population = cities[['Metropolitan area', 'Population']]

# Merge the names with the data of interest
df2 = pd.merge(teams_c, population, how = 'left', left_index = True, right_index = True)
df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
df2.reset_index(inplace = True)

# ------------- Match team with city -------------
# I crate two empty lists that I will fill with the indexes and the metropolitan area for each
# team
ind = []
val = []

# I pass the loop
for i in df.index:
    for j in range(0, len(df2)):
        for k in df2[[0, 1, 2]].iloc[j]:
            if i == k:
                val.append(df2['Metropolitan area'][j])
                ind.append(i)
                
# I create a new data frame with the teams + metropolitan area
df3 = pd.DataFrame(ind, val)
df3 = df3.rename(columns = {0 : 'team'})

# I reset the index for the dataframe with into of interest
df2 = df2.set_index('Metropolitan area')

# I merge and remove all unnecesary info
df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
df4 = df4[['team', 'Population']]

# Reset the index to merge the df later with the data from nhl_df
df4.reset_index(inplace = True)
df4.set_index('team', inplace = True)
df4

# Merge of the final dfs from cities and nhl_df
f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

# Some more cleaning
f_df.reset_index(inplace = True)
f_df = f_df.rename(columns = {'Matchname' : 'Team', 'index' : 'Area'})
f_df.set_index('Area', inplace = True)
f_df = f_df[['Team', 'W/L', 'Population']]

# I aggregate results by area
f_df['Pop2'] = [int(x) for x in f_df['Population']]
byArea = f_df.groupby('Area').agg({'W/L' : np.average, 'Pop2' : np.average})

# Create the lists 
population_by_region = list(byArea['Pop2'])
win_loss_by_region = list(byArea['W/L'])

# Calculate the correlation
corr = stats.pearsonr(population_by_region, win_loss_by_region)[0]
corr

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


0.012486162921209881

### - Solution -

In [21]:
def nhl_correlation(): 
    nhl_df = pd.read_csv('./Course1_Resources/assignments/assignment4/assets/nhl.csv')
    cities = pd.read_html('./Course1_Resources/assignments/assignment4/assets/wikipedia_data.html')[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # ------------- nhl_df -------------

    # --- Names cleaning ---
    # Pull the teams just fo 2018
    teams = nhl_df[nhl_df['year'] == 2018]['team']

    # Split the names for later match
    teams = teams.str.extractall('([\w]*)(\W)?\s?').unstack()[0]

    # Remove the NaN columns
    teams = teams[[0, 1, 2]]

    # Remove the 'Divisions'
    teams = teams[teams[1] != 'Division']

    # Replace NaN with empty strings
    teams = teams.replace(np.nan, '')

    # Create and append a list with the last name of the team for later match with cities df
    team = []
    index = []

    for ind, row in teams.iterrows():
        if row[2] != '':
            team.append(row[2])
            index.append(ind)
        else:
            team.append(row[1])
            index.append(ind)

    new = pd.DataFrame(team, index)
    n_teams = pd.merge(teams, new, how = 'left', left_index = True, right_index = True)
    n_teams.columns = [0, 1, 2, 'Matchname']
    n_teams

    # --- Putting together with the rest of the data ---
    # Recovering data from wins and losses
    wins_losses = nhl_df[nhl_df['year'] == 2018][['W', 'L']]
    wins_losses = wins_losses.drop([0, 9, 18, 26], axis = 0)

    # Merging both dataframes
    df = pd.merge(n_teams, wins_losses, how = 'left', left_index = True, right_index = True)

    # Convert columns W & L to numbers
    df['Wint'] = [int(x) for x in df['W']]
    df['Lint'] = [int(x) for x in df['L']]
    df['W/L'] = df['Wint'] / (df['Wint'] + df['Lint'])
    df.set_index('Matchname', inplace = True)


    # ------------- cities -------------
    # Pulling the names
    teams_c = cities['NHL'].str.extractall('([A-Z][a-z]+)').unstack()

    # Recovering population data
    # I rename the column for easier handle
    cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})
    # I store the data of interest in a new df: population
    population = cities[['Metropolitan area', 'Population']]

    # Merge the names with the data of interest
    df2 = pd.merge(teams_c, population, how = 'left', left_index = True, right_index = True)
    df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
    df2.reset_index(inplace = True)

    # ------------- Match team with city -------------
    # I crate two empty lists that I will fill with the indexes and the metropolitan area for each
    # team
    ind = []
    val = []

    # I pass the loop
    for i in df.index:
        for j in range(0, len(df2)):
            for k in df2[[0, 1, 2]].iloc[j]:
                if i == k:
                    val.append(df2['Metropolitan area'][j])
                    ind.append(i)

    # I create a new data frame with the teams + metropolitan area
    df3 = pd.DataFrame(ind, val)
    df3 = df3.rename(columns = {0 : 'team'})

    # I reset the index for the dataframe with into of interest
    df2 = df2.set_index('Metropolitan area')

    # I merge and remove all unnecesary info
    df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
    df4 = df4[['team', 'Population']]

    # Reset the index to merge the df later with the data from nhl_df
    df4.reset_index(inplace = True)
    df4.set_index('team', inplace = True)
    df4

    # Merge of the final dfs from cities and nhl_df
    f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

    # Some more cleaning
    f_df.reset_index(inplace = True)
    f_df = f_df.rename(columns = {'Matchname' : 'Team', 'index' : 'Area'})
    f_df.set_index('Area', inplace = True)
    f_df = f_df[['Team', 'W/L', 'Population']]

    # I aggregate results by area
    f_df['Pop2'] = [int(x) for x in f_df['Population']]
    byArea = f_df.groupby('Area').agg({'W/L' : np.average, 'Pop2' : np.average})

    # Create the lists 
    population_by_region = list(byArea['Pop2'])
    win_loss_by_region = list(byArea['W/L'])

    corr = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
    return corr

    raise NotImplementedError()

In [24]:
type(nhl_correlation()) == np.float64

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


True

# Question 2

In [8]:
# ------------- nba_df -------------
# Pull data for year of interest
teams = nba_df[nba_df['year'] == 2018][['team', 'W', 'L']]

# Take the last part of the names for later matching with cities df
lastnames = pd.Series([re.findall('(?<=\s)[\w]*', x)[-2] for x in teams['team']])
lastnames.name = 'lastnames'

# Adding the lastnames to our df
df = pd.merge(teams, lastnames, how = 'left', left_index = True, right_index = True)

# Calculate the W/L rate
df['Wint'] = [int(x) for x in df['W']]
df['Lint'] = [int(x) for x in df['L']]
df['W/L'] = df['Wint'] / (df['Wint'] + df['Lint'])
df.set_index('lastnames', inplace = True)


# ------------- cities -------------
# Pulling the names
teams_c = cities['NBA'].str.extractall('([A-Z][a-z]+)').unstack()

# Recovering population data
# I rename the column for easier handle
cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})

# I store the data of interest in a new df: population
population = cities[['Metropolitan area', 'Population', 'NBA']]

# Merge the names with the data of interest
df2 = pd.merge(population, teams_c, how = 'left', left_index = True, right_index = True)
df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
df2.reset_index(inplace = True)

# Replace values in Philadelphia as the regex didn't match this one
df2.at[6, 0] = '76ers'

# Transform the Population data to integer for later work
df2['Population'] = [int(x) for x in df2['Population']]

# ------------- Match team with city -------------
# I crate two empty lists that I will fill with the indexes and the metropolitan area for each
# team
ind = []
val = []

# I pass the loop
for i in df.index:
    for j in range(0, len(df2)):
        for k in df2[[0, 1]].iloc[j]:
            if i == k:
                val.append(df2['Metropolitan area'][j])
                ind.append(i)
                
# I create a new data frame with the teams + metropolitan area
df3 = pd.DataFrame(ind, val)
df3 = df3.rename(columns = {0 : 'team'})

# I reset the index for the dataframe with into of interest
df2 = df2.set_index('Metropolitan area')

# I merge and remove all unnecesary info
df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
df4 = df4[['team', 'Population']]

# Reset the index to merge the df later with the data from nhl_df
df4.reset_index(inplace = True)
df4.set_index('team', inplace = True)
df4

# Merge of the final dfs from cities and nhl_df
f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

# I aggregate results by area
byArea = f_df.groupby('index').agg({'W/L' : np.mean, 'Population' : np.mean})

# Create the lists 
population_by_region = list(byArea['Population'])
win_loss_by_region = list(byArea['W/L'])

corr = stats.pearsonr(population_by_region, win_loss_by_region)[0]
corr

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


-0.17657160252844611

### - Solution -

In [6]:
def nba_correlation():
    nba_df=pd.read_csv("./Course1_Resources/assignments/assignment4/assets/nba.csv")
    cities = pd.read_html("./Course1_Resources/assignments/assignment4/assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # ------------- nba_df -------------
    # Pull data for year of interest
    teams = nba_df[nba_df['year'] == 2018][['team', 'W', 'L']]

    # Take the last part of the names for later matching with cities df
    lastnames = pd.Series([re.findall('(?<=\s)[\w]*', x)[-2] for x in teams['team']])
    lastnames.name = 'lastnames'

    # Adding the lastnames to our df
    df = pd.merge(teams, lastnames, how = 'left', left_index = True, right_index = True)

    # Calculate the W/L rate
    df['Wint'] = [int(x) for x in df['W']]
    df['Lint'] = [int(x) for x in df['L']]
    df['W/L'] = df['Wint'] / (df['Wint'] + df['Lint'])
    df.set_index('lastnames', inplace = True)


    # ------------- cities -------------
    # Pulling the names
    teams_c = cities['NBA'].str.extractall('([A-Z][a-z]+)').unstack()

    # Recovering population data
    # I rename the column for easier handle
    cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})

    # I store the data of interest in a new df: population
    population = cities[['Metropolitan area', 'Population', 'NBA']]

    # Merge the names with the data of interest
    df2 = pd.merge(population, teams_c, how = 'left', left_index = True, right_index = True)
    df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
    df2.reset_index(inplace = True)

    # Replace values in Philadelphia as the regex didn't match this one
    df2.at[6, 0] = '76ers'

    # Transform the Population data to integer for later work
    df2['Population'] = [int(x) for x in df2['Population']]

    # ------------- Match team with city -------------
    # I crate two empty lists that I will fill with the indexes and the metropolitan area for each
    # team
    ind = []
    val = []

    # I pass the loop
    for i in df.index:
        for j in range(0, len(df2)):
            for k in df2[[0, 1]].iloc[j]:
                if i == k:
                    val.append(df2['Metropolitan area'][j])
                    ind.append(i)

    # I create a new data frame with the teams + metropolitan area
    df3 = pd.DataFrame(ind, val)
    df3 = df3.rename(columns = {0 : 'team'})

    # I reset the index for the dataframe with into of interest
    df2 = df2.set_index('Metropolitan area')

    # I merge and remove all unnecesary info
    df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
    df4 = df4[['team', 'Population']]

    # Reset the index to merge the df later with the data from nhl_df
    df4.reset_index(inplace = True)
    df4.set_index('team', inplace = True)
    df4

    # Merge of the final dfs from cities and nhl_df
    f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

    # I aggregate results by area
    byArea = f_df.groupby('index').agg({'W/L' : np.mean, 'Population' : np.mean})

    # Create the lists 
    population_by_region = list(byArea['Population'])
    win_loss_by_region = list(byArea['W/L'])

    corr = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
    return corr
    
    raise NotImplementedError()

In [7]:
nba_correlation()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


-0.17657160252844611

# Question 3

In [9]:
# ------------- mlb_df -------------
# Pull data for year of interest
teams = mlb_df[ mlb_df['year'] == 2018][['team', 'W-L%']]
teams = teams.rename(columns = {'W-L%' : 'W/L'})

# Take the last part of the names for later matching with cities df
teams['lastname'] = [re.findall('(?<=\s)[\w]*', x)[-1] for x in teams['team']]

# Replace values for the White and Red Sox, to avoid later on mismatching
# Red Sox -> RSox
teams.at[0, 'lastname'] = 'RSox'
# White Sox -> WSox
teams.at[8, 'lastname'] = 'WSox'
df = teams.set_index('lastname')
df

# ------------- cities -------------
# Pulling the names
teams_c = cities['MLB'].str.extractall('([A-Z][a-z]+)').unstack()
teams_c

# Recovering population data
# I rename the column for easier handle
cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})

# I store the data of interest in a new df: population
population = cities[['Metropolitan area', 'Population', 'MLB']]

# Merge the names with the data of interest
df2 = pd.merge(population, teams_c, how = 'left', left_index = True, right_index = True)
df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
df2.reset_index(inplace = True)

# Replace values for the White and Red Sox, to avoid later on mismatching
# White Sox -> WSox
df2.at[3, 2] = 'WSox'
# Red Sox -> RSow
df2.at[7, 1] = 'RSox'

# Transform the Population data to integer for later work
df2['Population'] = [int(x) for x in df2['Population']]
df2

# ------------- Match team with city -------------
# I crate two empty lists that I will fill with the indexes and the metropolitan area for each
# team
ind = []
val = []

# I pass the loop
for i in df.index:
    for j in range(0, len(df2)):
        for k in df2[[0, 1, 2]].iloc[j]:
            if i == k:
                val.append(df2['Metropolitan area'][j])
                ind.append(i)

# I create a new data frame with the teams + metropolitan area
df3 = pd.DataFrame(ind, val)
df3 = df3.rename(columns = {0 : 'team'})


# I reset the index for the dataframe with into of interest
df2 = df2.set_index('Metropolitan area')

# I merge and remove all unnecesary info
df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
df4 = df4[['team', 'Population']]

# Reset the index to merge the df later with the data from nhl_df
df4.reset_index(inplace = True)
df4.set_index('team', inplace = True)
df4

# Merge of the final dfs from cities and nhl_df
f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

# I aggregate results by area
byArea = f_df.groupby('index').agg({'W/L' : np.mean, 'Population' : np.mean})

# Create the lists 
population_by_region = list(byArea['Population'])
win_loss_by_region = list(byArea['W/L'])

corr = stats.pearsonr(population_by_region, win_loss_by_region)[0]
corr

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


0.15003737475409498

### - Solution -

In [12]:
def mlb_correlation(): 
    mlb_df=pd.read_csv("./Course1_Resources/assignments/assignment4/assets/mlb.csv")
    cities = pd.read_html("./Course1_Resources/assignments/assignment4/assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # ------------- mlb_df -------------
    # Pull data for year of interest
    teams = mlb_df[ mlb_df['year'] == 2018][['team', 'W-L%']]
    teams = teams.rename(columns = {'W-L%' : 'W/L'})

    # Take the last part of the names for later matching with cities df
    teams['lastname'] = [re.findall('(?<=\s)[\w]*', x)[-1] for x in teams['team']]

    # Replace values for the White and Red Sox, to avoid later on mismatching
    # Red Sox -> RSox
    teams.at[0, 'lastname'] = 'RSox'
    # White Sox -> WSox
    teams.at[8, 'lastname'] = 'WSox'
    df = teams.set_index('lastname')
    df

    # ------------- cities -------------
    # Pulling the names
    teams_c = cities['MLB'].str.extractall('([A-Z][a-z]+)').unstack()
    teams_c

    # Recovering population data
    # I rename the column for easier handle
    cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})

    # I store the data of interest in a new df: population
    population = cities[['Metropolitan area', 'Population', 'MLB']]

    # Merge the names with the data of interest
    df2 = pd.merge(population, teams_c, how = 'left', left_index = True, right_index = True)
    df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
    df2.reset_index(inplace = True)

    # Replace values for the White and Red Sox, to avoid later on mismatching
    # White Sox -> WSox
    df2.at[3, 2] = 'WSox'
    # Red Sox -> RSow
    df2.at[7, 1] = 'RSox'

    # Transform the Population data to integer for later work
    df2['Population'] = [int(x) for x in df2['Population']]
    df2

    # ------------- Match team with city -------------
    # I crate two empty lists that I will fill with the indexes and the metropolitan area for each
    # team
    ind = []
    val = []

    # I pass the loop
    for i in df.index:
        for j in range(0, len(df2)):
            for k in df2[[0, 1, 2]].iloc[j]:
                if i == k:
                    val.append(df2['Metropolitan area'][j])
                    ind.append(i)

    # I create a new data frame with the teams + metropolitan area
    df3 = pd.DataFrame(ind, val)
    df3 = df3.rename(columns = {0 : 'team'})


    # I reset the index for the dataframe with into of interest
    df2 = df2.set_index('Metropolitan area')

    # I merge and remove all unnecesary info
    df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
    df4 = df4[['team', 'Population']]

    # Reset the index to merge the df later with the data from nhl_df
    df4.reset_index(inplace = True)
    df4.set_index('team', inplace = True)
    df4

    # Merge of the final dfs from cities and nhl_df
    f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

    # I aggregate results by area
    byArea = f_df.groupby('index').agg({'W/L' : np.mean, 'Population' : np.mean})

    # Create the lists 
    population_by_region = list(byArea['Population'])
    win_loss_by_region = list(byArea['W/L'])

    corr = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
    return corr
    
    raise NotImplementedError()

In [14]:
mlb_correlation()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


0.15003737475409498

# Question 4

In [17]:
# ------------- nfl_df -------------
# Pull data for year of interest
teams = nfl_df[nfl_df['year'] == 2018][['team', 'W-L%']]

# Update name for later purpose
teams = teams.rename(columns = {'W-L%' : 'W/L'})

# Remove the rows with no team data
teams = teams.drop([0, 5, 10, 15, 20, 25, 30, 35])

# Cast object to float
teams['W/L'] = teams['W/L'].astype(float)

# Split the names for later match
teams['lastname'] = [re.findall('(?<=\s)[\w]*', x)[-1] for x in teams['team']]
df = teams.set_index('lastname')

# ------------- cities -------------
# Pulling the names
teams_c = cities['NFL'].str.extractall('([A-Z][a-z]+)').unstack()

# Recovering population data
# I rename the column for easier handle
cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})

# I store the data of interest in a new df: population
population = cities[['Metropolitan area', 'Population', 'NFL']]

# Merge the names with the data of interest
df2 = pd.merge(population, teams_c, how = 'left', left_index = True, right_index = True)
df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
df2.reset_index(inplace = True)

# Replace a regex' value mismatch
df2.at[2, 0] = '49ers'
df2.at[2, 1] = 'Raiders'

# Transform the Population data to integer for later work
df2['Population'] = [int(x) for x in df2['Population']]

# ------------- Match team with city -------------
# I crate two empty lists that I will fill with the indexes and the metropolitan area for each
# team
ind = []
val = []

# I pass the loop
for i in df.index:
    for j in range(0, len(df2)):
        for k in df2[[0, 1]].iloc[j]:
            if i == k:
                val.append(df2['Metropolitan area'][j])
                ind.append(i)

# I create a new data frame with the teams + metropolitan area
df3 = pd.DataFrame(ind, val)
df3 = df3.rename(columns = {0 : 'team'})

# I reset the index for the dataframe with into of interest
df2 = df2.set_index('Metropolitan area')

# I merge and remove all unnecesary info
df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
df4 = df4[['team', 'Population']]

# Reset the index to merge the df later with the data from nhl_df
df4.reset_index(inplace = True)
df4.set_index('team', inplace = True)
df4

# Merge of the final dfs from cities and nhl_df
f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

# I aggregate results by area
byArea = f_df.groupby('index').agg({'W/L' : np.mean, 'Population' : np.mean})

# Create the lists 
population_by_region = list(byArea['Population'])
win_loss_by_region = list(byArea['W/L'])

corr = stats.pearsonr(population_by_region, win_loss_by_region)[0]
corr

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


0.004282141436393022

### - Solution -

In [33]:
def nfl_correlation(): 
    nfl_df = pd.read_csv("./Course1_Resources/assignments/assignment4/assets/nfl.csv")
    cities = pd.read_html("./Course1_Resources/assignments/assignment4/assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # ------------- nfl_df -------------
    # Pull data for year of interest
    teams = nfl_df[nfl_df['year'] == 2018][['team', 'W-L%']]

    # Update name for later purpose
    teams = teams.rename(columns = {'W-L%' : 'W/L'})

    # Remove the rows with no team data
    teams = teams.drop([0, 5, 10, 15, 20, 25, 30, 35])

    # Cast object to float
    teams['W/L'] = teams['W/L'].astype(float)

    # Split the names for later match
    teams['lastname'] = [re.findall('(?<=\s)[\w]*', x)[-1] for x in teams['team']]
    df = teams.set_index('lastname')

    # ------------- cities -------------
    # Pulling the names
    teams_c = cities['NFL'].str.extractall('([A-Z][a-z]+)').unstack()

    # Recovering population data
    # I rename the column for easier handle
    cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})

    # I store the data of interest in a new df: population
    population = cities[['Metropolitan area', 'Population', 'NFL']]

    # Merge the names with the data of interest
    df2 = pd.merge(population, teams_c, how = 'left', left_index = True, right_index = True)
    df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
    df2.reset_index(inplace = True)

    # Replace a regex' value mismatch
    df2.at[2, 0] = '49ers'
    df2.at[2, 1] = 'Raiders'

    # Transform the Population data to integer for later work
    df2['Population'] = [int(x) for x in df2['Population']]

    # ------------- Match team with city -------------
    # I crate two empty lists that I will fill with the indexes and the metropolitan area for each
    # team
    ind = []
    val = []

    # I pass the loop
    for i in df.index:
        for j in range(0, len(df2)):
            for k in df2[[0, 1]].iloc[j]:
                if i == k:
                    val.append(df2['Metropolitan area'][j])
                    ind.append(i)

    # I create a new data frame with the teams + metropolitan area
    df3 = pd.DataFrame(ind, val)
    df3 = df3.rename(columns = {0 : 'team'})

    # I reset the index for the dataframe with into of interest
    df2 = df2.set_index('Metropolitan area')

    # I merge and remove all unnecesary info
    df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
    df4 = df4[['team', 'Population']]

    # Reset the index to merge the df later with the data from nhl_df
    df4.reset_index(inplace = True)
    df4.set_index('team', inplace = True)
    df4

    # Merge of the final dfs from cities and nhl_df
    f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

    # I aggregate results by area
    byArea = f_df.groupby('index').agg({'W/L' : np.mean, 'Population' : np.mean})

    # Create the lists 
    population_by_region = list(byArea['Population'])
    win_loss_by_region = list(byArea['W/L'])

    corr = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
    return corr
    
    raise NotImplementedError()

In [35]:
nfl_correlation()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


0.004282141436393022

# Question 5

In [2]:
# ------------- nhl_df -------------
def nhl_df(): 
    nhl_df = pd.read_csv('./Course1_Resources/assignments/assignment4/assets/nhl.csv')
    cities = pd.read_html('./Course1_Resources/assignments/assignment4/assets/wikipedia_data.html')[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # ------------- nhl_df -------------

    # --- Names cleaning ---
    # Pull the teams just fo 2018
    teams = nhl_df[nhl_df['year'] == 2018]['team']

    # Split the names for later match
    teams = teams.str.extractall('([\w]*)(\W)?\s?').unstack()[0]

    # Remove the NaN columns
    teams = teams[[0, 1, 2]]

    # Remove the 'Divisions'
    teams = teams[teams[1] != 'Division']

    # Replace NaN with empty strings
    teams = teams.replace(np.nan, '')

    # Create and append a list with the last name of the team for later match with cities df
    team = []
    index = []

    for ind, row in teams.iterrows():
        if row[2] != '':
            team.append(row[2])
            index.append(ind)
        else:
            team.append(row[1])
            index.append(ind)

    new = pd.DataFrame(team, index)
    n_teams = pd.merge(teams, new, how = 'left', left_index = True, right_index = True)
    n_teams.columns = [0, 1, 2, 'Matchname']
    n_teams

    # --- Putting together with the rest of the data ---
    # Recovering data from wins and losses
    wins_losses = nhl_df[nhl_df['year'] == 2018][['W', 'L']]
    wins_losses = wins_losses.drop([0, 9, 18, 26], axis = 0)

    # Merging both dataframes
    df = pd.merge(n_teams, wins_losses, how = 'left', left_index = True, right_index = True)

    # Convert columns W & L to numbers
    df['Wint'] = [int(x) for x in df['W']]
    df['Lint'] = [int(x) for x in df['L']]
    df['W/L'] = df['Wint'] / (df['Wint'] + df['Lint'])
    df.set_index('Matchname', inplace = True)


    # ------------- cities -------------
    # Pulling the names
    teams_c = cities['NHL'].str.extractall('([A-Z][a-z]+)').unstack()

    # Recovering population data
    # I rename the column for easier handle
    cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})
    # I store the data of interest in a new df: population
    population = cities[['Metropolitan area', 'Population']]

    # Merge the names with the data of interest
    df2 = pd.merge(teams_c, population, how = 'left', left_index = True, right_index = True)
    df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
    df2.reset_index(inplace = True)

    # ------------- Match team with city -------------
    # I crate two empty lists that I will fill with the indexes and the metropolitan area for each
    # team
    ind = []
    val = []

    # I pass the loop
    for i in df.index:
        for j in range(0, len(df2)):
            for k in df2[[0, 1, 2]].iloc[j]:
                if i == k:
                    val.append(df2['Metropolitan area'][j])
                    ind.append(i)

    # I create a new data frame with the teams + metropolitan area
    df3 = pd.DataFrame(ind, val)
    df3 = df3.rename(columns = {0 : 'team'})

    # I reset the index for the dataframe with into of interest
    df2 = df2.set_index('Metropolitan area')

    # I merge and remove all unnecesary info
    df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
    df4 = df4[['team', 'Population']]

    # Reset the index to merge the df later with the data from nhl_df
    df4.reset_index(inplace = True)
    df4.set_index('team', inplace = True)
    df4

    # Merge of the final dfs from cities and nhl_df
    f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

    # Some more cleaning
    f_df.reset_index(inplace = True)
    f_df = f_df.rename(columns = {'Matchname' : 'Team', 'index' : 'Area'})
    f_df.set_index('Area', inplace = True)
    f_df = f_df[['Team', 'W/L', 'Population']]

    # I aggregate results by area
    f_df['Pop2'] = [int(x) for x in f_df['Population']]
    byArea = f_df.groupby('Area').agg({'W/L' : np.average, 'Pop2' : np.average})
    
    return byArea

    raise NotImplementedError()
    
    
# ------------- nba_df -------------
def nba_df():
    nba_df=pd.read_csv("./Course1_Resources/assignments/assignment4/assets/nba.csv")
    cities = pd.read_html("./Course1_Resources/assignments/assignment4/assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # ------------- nba_df -------------
    # Pull data for year of interest
    teams = nba_df[nba_df['year'] == 2018][['team', 'W', 'L']]

    # Take the last part of the names for later matching with cities df
    lastnames = pd.Series([re.findall('(?<=\s)[\w]*', x)[-2] for x in teams['team']])
    lastnames.name = 'lastnames'

    # Adding the lastnames to our df
    df = pd.merge(teams, lastnames, how = 'left', left_index = True, right_index = True)

    # Calculate the W/L rate
    df['Wint'] = [int(x) for x in df['W']]
    df['Lint'] = [int(x) for x in df['L']]
    df['W/L'] = df['Wint'] / (df['Wint'] + df['Lint'])
    df.set_index('lastnames', inplace = True)


    # ------------- cities -------------
    # Pulling the names
    teams_c = cities['NBA'].str.extractall('([A-Z][a-z]+)').unstack()

    # Recovering population data
    # I rename the column for easier handle
    cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})

    # I store the data of interest in a new df: population
    population = cities[['Metropolitan area', 'Population', 'NBA']]

    # Merge the names with the data of interest
    df2 = pd.merge(population, teams_c, how = 'left', left_index = True, right_index = True)
    df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
    df2.reset_index(inplace = True)

    # Replace values in Philadelphia as the regex didn't match this one
    df2.at[6, 0] = '76ers'

    # Transform the Population data to integer for later work
    df2['Population'] = [int(x) for x in df2['Population']]

    # ------------- Match team with city -------------
    # I crate two empty lists that I will fill with the indexes and the metropolitan area for each
    # team
    ind = []
    val = []

    # I pass the loop
    for i in df.index:
        for j in range(0, len(df2)):
            for k in df2[[0, 1]].iloc[j]:
                if i == k:
                    val.append(df2['Metropolitan area'][j])
                    ind.append(i)

    # I create a new data frame with the teams + metropolitan area
    df3 = pd.DataFrame(ind, val)
    df3 = df3.rename(columns = {0 : 'team'})

    # I reset the index for the dataframe with into of interest
    df2 = df2.set_index('Metropolitan area')

    # I merge and remove all unnecesary info
    df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
    df4 = df4[['team', 'Population']]

    # Reset the index to merge the df later with the data from nhl_df
    df4.reset_index(inplace = True)
    df4.set_index('team', inplace = True)
    df4

    # Merge of the final dfs from cities and nhl_df
    f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

    # I aggregate results by area
    byArea = f_df.groupby('index').agg({'W/L' : np.mean, 'Population' : np.mean})
    
    return byArea
    
    raise NotImplementedError()
    
    
# ------------- mlb_df -------------
def mlb_df(): 
    mlb_df=pd.read_csv("./Course1_Resources/assignments/assignment4/assets/mlb.csv")
    cities = pd.read_html("./Course1_Resources/assignments/assignment4/assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # ------------- mlb_df -------------
    # Pull data for year of interest
    teams = mlb_df[ mlb_df['year'] == 2018][['team', 'W-L%']]
    teams = teams.rename(columns = {'W-L%' : 'W/L'})

    # Take the last part of the names for later matching with cities df
    teams['lastname'] = [re.findall('(?<=\s)[\w]*', x)[-1] for x in teams['team']]

    # Replace values for the White and Red Sox, to avoid later on mismatching
    # Red Sox -> RSox
    teams.at[0, 'lastname'] = 'RSox'
    # White Sox -> WSox
    teams.at[8, 'lastname'] = 'WSox'
    df = teams.set_index('lastname')
    df

    # ------------- cities -------------
    # Pulling the names
    teams_c = cities['MLB'].str.extractall('([A-Z][a-z]+)').unstack()
    teams_c

    # Recovering population data
    # I rename the column for easier handle
    cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})

    # I store the data of interest in a new df: population
    population = cities[['Metropolitan area', 'Population', 'MLB']]

    # Merge the names with the data of interest
    df2 = pd.merge(population, teams_c, how = 'left', left_index = True, right_index = True)
    df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
    df2.reset_index(inplace = True)

    # Replace values for the White and Red Sox, to avoid later on mismatching
    # White Sox -> WSox
    df2.at[3, 2] = 'WSox'
    # Red Sox -> RSow
    df2.at[7, 1] = 'RSox'

    # Transform the Population data to integer for later work
    df2['Population'] = [int(x) for x in df2['Population']]
    df2

    # ------------- Match team with city -------------
    # I crate two empty lists that I will fill with the indexes and the metropolitan area for each
    # team
    ind = []
    val = []

    # I pass the loop
    for i in df.index:
        for j in range(0, len(df2)):
            for k in df2[[0, 1, 2]].iloc[j]:
                if i == k:
                    val.append(df2['Metropolitan area'][j])
                    ind.append(i)

    # I create a new data frame with the teams + metropolitan area
    df3 = pd.DataFrame(ind, val)
    df3 = df3.rename(columns = {0 : 'team'})


    # I reset the index for the dataframe with into of interest
    df2 = df2.set_index('Metropolitan area')

    # I merge and remove all unnecesary info
    df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
    df4 = df4[['team', 'Population']]

    # Reset the index to merge the df later with the data from nhl_df
    df4.reset_index(inplace = True)
    df4.set_index('team', inplace = True)
    df4

    # Merge of the final dfs from cities and nhl_df
    f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

    # I aggregate results by area
    byArea = f_df.groupby('index').agg({'W/L' : np.mean, 'Population' : np.mean})

    return byArea
    
    raise NotImplementedError()
   
# ------------- nfl_df -------------
def nfl_df(): 
    nfl_df = pd.read_csv("./Course1_Resources/assignments/assignment4/assets/nfl.csv")
    cities = pd.read_html("./Course1_Resources/assignments/assignment4/assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    # ------------- nfl_df -------------
    # Pull data for year of interest
    teams = nfl_df[nfl_df['year'] == 2018][['team', 'W-L%']]

    # Update name for later purpose
    teams = teams.rename(columns = {'W-L%' : 'W/L'})

    # Remove the rows with no team data
    teams = teams.drop([0, 5, 10, 15, 20, 25, 30, 35])

    # Cast object to float
    teams['W/L'] = teams['W/L'].astype(float)

    # Split the names for later match
    teams['lastname'] = [re.findall('(?<=\s)[\w]*', x)[-1] for x in teams['team']]
    df = teams.set_index('lastname')

    # ------------- cities -------------
    # Pulling the names
    teams_c = cities['NFL'].str.extractall('([A-Z][a-z]+)').unstack()

    # Recovering population data
    # I rename the column for easier handle
    cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})

    # I store the data of interest in a new df: population
    population = cities[['Metropolitan area', 'Population', 'NFL']]

    # Merge the names with the data of interest
    df2 = pd.merge(population, teams_c, how = 'left', left_index = True, right_index = True)
    df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
    df2.reset_index(inplace = True)

    # Replace a regex' value mismatch
    df2.at[2, 0] = '49ers'
    df2.at[2, 1] = 'Raiders'

    # Transform the Population data to integer for later work
    df2['Population'] = [int(x) for x in df2['Population']]

    # ------------- Match team with city -------------
    # I crate two empty lists that I will fill with the indexes and the metropolitan area for each
    # team
    ind = []
    val = []

    # I pass the loop
    for i in df.index:
        for j in range(0, len(df2)):
            for k in df2[[0, 1]].iloc[j]:
                if i == k:
                    val.append(df2['Metropolitan area'][j])
                    ind.append(i)

    # I create a new data frame with the teams + metropolitan area
    df3 = pd.DataFrame(ind, val)
    df3 = df3.rename(columns = {0 : 'team'})

    # I reset the index for the dataframe with into of interest
    df2 = df2.set_index('Metropolitan area')

    # I merge and remove all unnecesary info
    df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
    df4 = df4[['team', 'Population']]

    # Reset the index to merge the df later with the data from nhl_df
    df4.reset_index(inplace = True)
    df4.set_index('team', inplace = True)
    df4

    # Merge of the final dfs from cities and nhl_df
    f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

    # I aggregate results by area
    byArea = f_df.groupby('index').agg({'W/L' : np.mean, 'Population' : np.mean})

    return byArea
    
    raise NotImplementedError()
    
    
# ------------- Correlations -------------
nhl = nhl_df()
nba = nba_df()
mlb = mlb_df()
nfl = nfl_df()

df = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
df = df.rename(columns ={'Metropolitan area' : 'Area',
                               'Population (2016 est.)[8]' : 'Population'})
df.set_index('Area', inplace = True)

df = pd.merge(df, nhl['W/L'], how = 'left', left_index = True, right_index = True)
df = df.rename(columns = {'W/L' : 'NHL'})
df = pd.merge(df, nba['W/L'], how = 'left', left_index = True, right_index = True)
df = df.rename(columns = {'W/L' : 'NBA'})
df = pd.merge(df, mlb['W/L'], how = 'left', left_index = True, right_index = True)
df = df.rename(columns = {'W/L' : 'MLB'})
df = pd.merge(df, nfl['W/L'], how = 'left', left_index = True, right_index = True)
df = df.rename(columns = {'W/L' : 'NFL'})

sports = ['NFL', 'NBA', 'NHL', 'MLB']

p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

for i in sports:
    for j in sports:
        if i != j:
            matrix = df[[i, j]].dropna()
            p_values.at[i, j] = stats.ttest_rel(matrix[i], matrix[j])[1]
        
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.937445,0.030318,0.803459
NBA,0.937445,NaN,0.022297,0.951629
NHL,0.030318,0.022297,NaN,0.000703
MLB,0.803459,0.951629,0.000703,NaN


### - Solution -

In [3]:
def sports_team_performance():

    # ------------- nhl_df -------------
    def nhl_df(): 
        nhl_df = pd.read_csv('./Course1_Resources/assignments/assignment4/assets/nhl.csv')
        cities = pd.read_html('./Course1_Resources/assignments/assignment4/assets/wikipedia_data.html')[1]
        cities = cities.iloc[:-1,[0,3,5,6,7,8]]

        # ------------- nhl_df -------------

        # --- Names cleaning ---
        # Pull the teams just fo 2018
        teams = nhl_df[nhl_df['year'] == 2018]['team']

        # Split the names for later match
        teams = teams.str.extractall('([\w]*)(\W)?\s?').unstack()[0]

        # Remove the NaN columns
        teams = teams[[0, 1, 2]]

        # Remove the 'Divisions'
        teams = teams[teams[1] != 'Division']

        # Replace NaN with empty strings
        teams = teams.replace(np.nan, '')

        # Create and append a list with the last name of the team for later match with cities df
        team = []
        index = []

        for ind, row in teams.iterrows():
            if row[2] != '':
                team.append(row[2])
                index.append(ind)
            else:
                team.append(row[1])
                index.append(ind)

        new = pd.DataFrame(team, index)
        n_teams = pd.merge(teams, new, how = 'left', left_index = True, right_index = True)
        n_teams.columns = [0, 1, 2, 'Matchname']
        n_teams

        # --- Putting together with the rest of the data ---
        # Recovering data from wins and losses
        wins_losses = nhl_df[nhl_df['year'] == 2018][['W', 'L']]
        wins_losses = wins_losses.drop([0, 9, 18, 26], axis = 0)

        # Merging both dataframes
        df = pd.merge(n_teams, wins_losses, how = 'left', left_index = True, right_index = True)

        # Convert columns W & L to numbers
        df['Wint'] = [int(x) for x in df['W']]
        df['Lint'] = [int(x) for x in df['L']]
        df['W/L'] = df['Wint'] / (df['Wint'] + df['Lint'])
        df.set_index('Matchname', inplace = True)


        # ------------- cities -------------
        # Pulling the names
        teams_c = cities['NHL'].str.extractall('([A-Z][a-z]+)').unstack()

        # Recovering population data
        # I rename the column for easier handle
        cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})
        # I store the data of interest in a new df: population
        population = cities[['Metropolitan area', 'Population']]

        # Merge the names with the data of interest
        df2 = pd.merge(teams_c, population, how = 'left', left_index = True, right_index = True)
        df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
        df2.reset_index(inplace = True)

        # ------------- Match team with city -------------
        # I crate two empty lists that I will fill with the indexes and the metropolitan area for each
        # team
        ind = []
        val = []

        # I pass the loop
        for i in df.index:
            for j in range(0, len(df2)):
                for k in df2[[0, 1, 2]].iloc[j]:
                    if i == k:
                        val.append(df2['Metropolitan area'][j])
                        ind.append(i)

        # I create a new data frame with the teams + metropolitan area
        df3 = pd.DataFrame(ind, val)
        df3 = df3.rename(columns = {0 : 'team'})

        # I reset the index for the dataframe with into of interest
        df2 = df2.set_index('Metropolitan area')

        # I merge and remove all unnecesary info
        df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
        df4 = df4[['team', 'Population']]

        # Reset the index to merge the df later with the data from nhl_df
        df4.reset_index(inplace = True)
        df4.set_index('team', inplace = True)
        df4

        # Merge of the final dfs from cities and nhl_df
        f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

        # Some more cleaning
        f_df.reset_index(inplace = True)
        f_df = f_df.rename(columns = {'Matchname' : 'Team', 'index' : 'Area'})
        f_df.set_index('Area', inplace = True)
        f_df = f_df[['Team', 'W/L', 'Population']]

        # I aggregate results by area
        f_df['Pop2'] = [int(x) for x in f_df['Population']]
        byArea = f_df.groupby('Area').agg({'W/L' : np.average, 'Pop2' : np.average})

        return byArea

        raise NotImplementedError()


    # ------------- nba_df -------------
    def nba_df():
        nba_df=pd.read_csv("./Course1_Resources/assignments/assignment4/assets/nba.csv")
        cities = pd.read_html("./Course1_Resources/assignments/assignment4/assets/wikipedia_data.html")[1]
        cities = cities.iloc[:-1,[0,3,5,6,7,8]]

        # ------------- nba_df -------------
        # Pull data for year of interest
        teams = nba_df[nba_df['year'] == 2018][['team', 'W', 'L']]

        # Take the last part of the names for later matching with cities df
        lastnames = pd.Series([re.findall('(?<=\s)[\w]*', x)[-2] for x in teams['team']])
        lastnames.name = 'lastnames'

        # Adding the lastnames to our df
        df = pd.merge(teams, lastnames, how = 'left', left_index = True, right_index = True)

        # Calculate the W/L rate
        df['Wint'] = [int(x) for x in df['W']]
        df['Lint'] = [int(x) for x in df['L']]
        df['W/L'] = df['Wint'] / (df['Wint'] + df['Lint'])
        df.set_index('lastnames', inplace = True)


        # ------------- cities -------------
        # Pulling the names
        teams_c = cities['NBA'].str.extractall('([A-Z][a-z]+)').unstack()

        # Recovering population data
        # I rename the column for easier handle
        cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})

        # I store the data of interest in a new df: population
        population = cities[['Metropolitan area', 'Population', 'NBA']]

        # Merge the names with the data of interest
        df2 = pd.merge(population, teams_c, how = 'left', left_index = True, right_index = True)
        df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
        df2.reset_index(inplace = True)

        # Replace values in Philadelphia as the regex didn't match this one
        df2.at[6, 0] = '76ers'

        # Transform the Population data to integer for later work
        df2['Population'] = [int(x) for x in df2['Population']]

        # ------------- Match team with city -------------
        # I crate two empty lists that I will fill with the indexes and the metropolitan area for each
        # team
        ind = []
        val = []

        # I pass the loop
        for i in df.index:
            for j in range(0, len(df2)):
                for k in df2[[0, 1]].iloc[j]:
                    if i == k:
                        val.append(df2['Metropolitan area'][j])
                        ind.append(i)

        # I create a new data frame with the teams + metropolitan area
        df3 = pd.DataFrame(ind, val)
        df3 = df3.rename(columns = {0 : 'team'})

        # I reset the index for the dataframe with into of interest
        df2 = df2.set_index('Metropolitan area')

        # I merge and remove all unnecesary info
        df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
        df4 = df4[['team', 'Population']]

        # Reset the index to merge the df later with the data from nhl_df
        df4.reset_index(inplace = True)
        df4.set_index('team', inplace = True)
        df4

        # Merge of the final dfs from cities and nhl_df
        f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

        # I aggregate results by area
        byArea = f_df.groupby('index').agg({'W/L' : np.mean, 'Population' : np.mean})

        return byArea

        raise NotImplementedError()


    # ------------- mlb_df -------------
    def mlb_df(): 
        mlb_df=pd.read_csv("./Course1_Resources/assignments/assignment4/assets/mlb.csv")
        cities = pd.read_html("./Course1_Resources/assignments/assignment4/assets/wikipedia_data.html")[1]
        cities = cities.iloc[:-1,[0,3,5,6,7,8]]

        # ------------- mlb_df -------------
        # Pull data for year of interest
        teams = mlb_df[ mlb_df['year'] == 2018][['team', 'W-L%']]
        teams = teams.rename(columns = {'W-L%' : 'W/L'})

        # Take the last part of the names for later matching with cities df
        teams['lastname'] = [re.findall('(?<=\s)[\w]*', x)[-1] for x in teams['team']]

        # Replace values for the White and Red Sox, to avoid later on mismatching
        # Red Sox -> RSox
        teams.at[0, 'lastname'] = 'RSox'
        # White Sox -> WSox
        teams.at[8, 'lastname'] = 'WSox'
        df = teams.set_index('lastname')
        df

        # ------------- cities -------------
        # Pulling the names
        teams_c = cities['MLB'].str.extractall('([A-Z][a-z]+)').unstack()
        teams_c

        # Recovering population data
        # I rename the column for easier handle
        cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})

        # I store the data of interest in a new df: population
        population = cities[['Metropolitan area', 'Population', 'MLB']]

        # Merge the names with the data of interest
        df2 = pd.merge(population, teams_c, how = 'left', left_index = True, right_index = True)
        df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
        df2.reset_index(inplace = True)

        # Replace values for the White and Red Sox, to avoid later on mismatching
        # White Sox -> WSox
        df2.at[3, 2] = 'WSox'
        # Red Sox -> RSow
        df2.at[7, 1] = 'RSox'

        # Transform the Population data to integer for later work
        df2['Population'] = [int(x) for x in df2['Population']]
        df2

        # ------------- Match team with city -------------
        # I crate two empty lists that I will fill with the indexes and the metropolitan area for each
        # team
        ind = []
        val = []

        # I pass the loop
        for i in df.index:
            for j in range(0, len(df2)):
                for k in df2[[0, 1, 2]].iloc[j]:
                    if i == k:
                        val.append(df2['Metropolitan area'][j])
                        ind.append(i)

        # I create a new data frame with the teams + metropolitan area
        df3 = pd.DataFrame(ind, val)
        df3 = df3.rename(columns = {0 : 'team'})


        # I reset the index for the dataframe with into of interest
        df2 = df2.set_index('Metropolitan area')

        # I merge and remove all unnecesary info
        df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
        df4 = df4[['team', 'Population']]

        # Reset the index to merge the df later with the data from nhl_df
        df4.reset_index(inplace = True)
        df4.set_index('team', inplace = True)
        df4

        # Merge of the final dfs from cities and nhl_df
        f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

        # I aggregate results by area
        byArea = f_df.groupby('index').agg({'W/L' : np.mean, 'Population' : np.mean})

        return byArea

        raise NotImplementedError()

    # ------------- nfl_df -------------
    def nfl_df(): 
        nfl_df = pd.read_csv("./Course1_Resources/assignments/assignment4/assets/nfl.csv")
        cities = pd.read_html("./Course1_Resources/assignments/assignment4/assets/wikipedia_data.html")[1]
        cities = cities.iloc[:-1,[0,3,5,6,7,8]]

        # ------------- nfl_df -------------
        # Pull data for year of interest
        teams = nfl_df[nfl_df['year'] == 2018][['team', 'W-L%']]

        # Update name for later purpose
        teams = teams.rename(columns = {'W-L%' : 'W/L'})

        # Remove the rows with no team data
        teams = teams.drop([0, 5, 10, 15, 20, 25, 30, 35])

        # Cast object to float
        teams['W/L'] = teams['W/L'].astype(float)

        # Split the names for later match
        teams['lastname'] = [re.findall('(?<=\s)[\w]*', x)[-1] for x in teams['team']]
        df = teams.set_index('lastname')

        # ------------- cities -------------
        # Pulling the names
        teams_c = cities['NFL'].str.extractall('([A-Z][a-z]+)').unstack()

        # Recovering population data
        # I rename the column for easier handle
        cities = cities.rename(columns = {'Population (2016 est.)[8]' : 'Population'})

        # I store the data of interest in a new df: population
        population = cities[['Metropolitan area', 'Population', 'NFL']]

        # Merge the names with the data of interest
        df2 = pd.merge(population, teams_c, how = 'left', left_index = True, right_index = True)
        df2 = df2.rename(columns = {(0, 0) : 0, (0, 1) : 1, (0, 2) : 2})
        df2.reset_index(inplace = True)

        # Replace a regex' value mismatch
        df2.at[2, 0] = '49ers'
        df2.at[2, 1] = 'Raiders'

        # Transform the Population data to integer for later work
        df2['Population'] = [int(x) for x in df2['Population']]

        # ------------- Match team with city -------------
        # I crate two empty lists that I will fill with the indexes and the metropolitan area for each
        # team
        ind = []
        val = []

        # I pass the loop
        for i in df.index:
            for j in range(0, len(df2)):
                for k in df2[[0, 1]].iloc[j]:
                    if i == k:
                        val.append(df2['Metropolitan area'][j])
                        ind.append(i)

        # I create a new data frame with the teams + metropolitan area
        df3 = pd.DataFrame(ind, val)
        df3 = df3.rename(columns = {0 : 'team'})

        # I reset the index for the dataframe with into of interest
        df2 = df2.set_index('Metropolitan area')

        # I merge and remove all unnecesary info
        df4 = pd.merge(df3, df2, how = 'left', left_index = True, right_index = True)
        df4 = df4[['team', 'Population']]

        # Reset the index to merge the df later with the data from nhl_df
        df4.reset_index(inplace = True)
        df4.set_index('team', inplace = True)
        df4

        # Merge of the final dfs from cities and nhl_df
        f_df = pd.merge(df, df4, how = 'left', left_index = True, right_index = True)

        # I aggregate results by area
        byArea = f_df.groupby('index').agg({'W/L' : np.mean, 'Population' : np.mean})

        return byArea

        raise NotImplementedError()


    # ------------- Correlations -------------
    nhl = nhl_df()
    nba = nba_df()
    mlb = mlb_df()
    nfl = nfl_df()

    df = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
    df = df.rename(columns ={'Metropolitan area' : 'Area',
                                   'Population (2016 est.)[8]' : 'Population'})
    df.set_index('Area', inplace = True)

    df = pd.merge(df, nhl['W/L'], how = 'left', left_index = True, right_index = True)
    df = df.rename(columns = {'W/L' : 'NHL'})
    df = pd.merge(df, nba['W/L'], how = 'left', left_index = True, right_index = True)
    df = df.rename(columns = {'W/L' : 'NBA'})
    df = pd.merge(df, mlb['W/L'], how = 'left', left_index = True, right_index = True)
    df = df.rename(columns = {'W/L' : 'MLB'})
    df = pd.merge(df, nfl['W/L'], how = 'left', left_index = True, right_index = True)
    df = df.rename(columns = {'W/L' : 'NFL'})

    sports = ['NFL', 'NBA', 'NHL', 'MLB']

    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

    for i in sports:
        for j in sports:
            if i != j:
                matrix = df[[i, j]].dropna()
                p_values.at[i, j] = stats.ttest_rel(matrix[i], matrix[j])[1]

    p_values
    
    return p_values

    raise NotImplementedError()

In [4]:
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937445,0.030318,0.803459
NBA,0.937445,NaN,0.022297,0.951629
NHL,0.030318,0.022297,NaN,0.000703
MLB,0.803459,0.951629,0.000703,NaN
